In [ ]:
import math
from collections import defaultdict
import json
import csv
import nltk

In [ ]:
userTweets = defaultdict(list)
# Read in the data
with open("/Users/chloelarkin/Desktop/maria_extras.json") as fin:
    for line in fin:
        json_data = json.loads(line)
        user = json_data['user']['id'] # Get the user ID
        tweet_id = json_data['text'] # Get the text of the tweet
        userTweets[user].append(tweet_id)
vocabulary = defaultdict(int)
documents = defaultdict(list)

In [ ]:
# Build global vocabulary and frequency tallies
for user in userTweets.keys(): # Build global vocabulary from all tweets
    for tweet in userTweets[user]:
        words = nltk.word_tokenize(tweet)
        for word in words:
            word = word.lower()
            vocabulary[word] += 1 # Build a global frequency tally for unigrams
print(len(vocabulary))

In [ ]:
# Start dictionaries for IDF feature vector and a dictionary of dictionaries of TF feature vectors (one per user)
idf_vals = dict.fromkeys(vocabulary.keys(), [])
all_tf_vals = defaultdict(list)

In [ ]:
# Calculate TF values for all individual users and extract info for IDF
for user in userTweets.keys(): # Now, build vocabulary for each user's doc (all tweets --> one doc here)
    userDocFreq = defaultdict(int)
    for key in vocabulary.keys():
        userDocFreq[key] = 0
    tf_vals = userDocFreq
    for tweet in userTweets[user]: #TODO: if we are filtering by date (before the labeled tweet), add that line here
        words = nltk.word_tokenize(tweet) # Tokenize all words in the tweet
        for word in words:
            word = word.lower() # Standardize word case (lowercase)
            userDocFreq[word] += 1
    documents[user] = userDocFreq
    for word in userDocFreq.copy():
        tf_vals[word] = (userDocFreq[word] / len(userDocFreq))
    all_tf_vals[user] = tf_vals

In [ ]:
# Calculate IDF values
idf_vals = defaultdict(int)
for key in vocabulary.keys():
    idf_vals[key] = 0
for word in vocabulary.copy(): # The IDF calculation:
    denom = 0
    for doc in documents: # Calculating the denominator for IDF. Clunky but hey, it works!
        if documents[doc][word] > 0:
            denom += 1
    idf_vals[word] = math.log(len(documents) / denom) # verify this?

In [ ]:
# Some sanity tests for the IDF value of common words:
print(idf_vals['rt'])
print(idf_vals['maria'])

# ...And not so common words:
print(idf_vals['elephant'])
print(idf_vals['joyful'])

In [ ]:
#TODO: part 2 - match up the extracted user histories with annotations

# Establish dicts of annotated scores for each tweet ID
# Keys = tweet IDs; Values = scores. Need to match these up with the user IDs. 
text_informative = defaultdict(str) #4000 scores
text_humanitarian = defaultdict(str) # 4000 scores
image_informative = defaultdict(str) # 4000 scores
image_humanitarian = defaultdict(str) # 4000 scores

# Read in the annotated remarks - Haven't integrated this yet
with open("/Users/chloelarkin/Desktop/hurricane_maria_final_data.tsv") as fin:
    reader = csv.DictReader(fin, dialect='excel-tab')
    print(reader.fieldnames)
    for line in reader:
        text_informative[line['tweet_id']]= line['text_info']
        text_humanitarian[line['tweet_id']]=line['text_human']
        image_informative[line['tweet_id']]=line['image_info']
        image_humanitarian[line['tweet_id']]=line['image_human']
